In [ ]:
import numpy as np 
import pandas as pd 
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import tensorflow as tf
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors    

In [ ]:
df = pd.read_csv("C:/PROJECTS/Abstractive Text Summarization/dataset/cnn_dailymail/train.csv") 
df.head()

In [ ]:
print(df.info())  
print(df.isnull().sum()) 
df.shape

In [ ]:
df = df.drop(['id'],axis=1)

In [ ]:
for i in range(5):
    print("Review #",i+1)
    print('Original Text : ')
    print(df.article[i])
   
    print('\n\nSummary Text : ')
    print(df.highlights[i])
    
    print('________________________________________________________________________________________\n\n')
     

Data Pre-processing

In [ ]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [ ]:
clean_highlights = []
for i in df.highlights:
    clean_highlights.append(clean_text(i, remove_stopwords=False))
print("Summaries are complete.")

clean_article = []
for k in df.article:
    clean_article.append(clean_text(k))
print("Texts are complete.")

In [ ]:
for i in range(5):
    print("Clean Review #",i+1)
    print(clean_highlights[i])
    print(clean_article[i])
    print()

In [ ]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [ ]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}
count_words(word_counts, clean_highlights)
count_words(word_counts, clean_article)          
print("Size of Vocabulary:", len(word_counts))

In [ ]:
import gzip


# (https://github.com/commonsense/conceptnet-numberbatch)

embeddings_index = {}
with gzip.open('C:/PROJECTS/numberbatch-en.txt.gz', mode='rt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))


In [ ]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

In [ ]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["","","",""]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

In [ ]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        if i < len(word_embedding_matrix):
            word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        if i < len(word_embedding_matrix):
            new_embedding = np.array(np.random.uniform(1.0, embedding_dim))
            word_embedding_matrix[i] = new_embedding


# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

In [ ]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int[""])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int[""])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [ ]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_highlights, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_article, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

In [ ]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [ ]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

In [ ]:
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))


In [ ]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

In [ ]:

def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    count_unk = 0
    for i in sentence:
      if i == vocab_to_int['']:
        count_unk += 1
    return count_unk

In [ ]:
sorted_summaries = []
sorted_texts = []
max_text_length = 959
max_summary_length = 116
min_length = 8
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
        
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

In [ ]:
import tensorflow as tf

def model_inputs():
    """Create keras for inputs to the model"""
    
    input_data = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input')
    targets = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='targets')
    lr = tf.keras.layers.Input(shape=(), dtype=tf.float32, name='learning_rate')
    keep_prob = tf.keras.layers.Input(shape=(), dtype=tf.float32, name='keep_prob')
    summary_length = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length


In [ ]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    """Remove the last word id from each batch and concatenate it to the beginning of each batch"""
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    start_token = tf.constant([vocab_to_int['']], dtype=tf.int32)
    dec_input = tf.concat([tf.fill([batch_size, 1], start_token), ending], axis=1)

    return dec_input

In [ ]:
import tensorflow as tf

def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    """Create the encoding layer"""
    
    encoder_cells = []
    for layer in range(num_layers):
        with tf.name_scope(f'encoder_{layer}'):
            cell_fw = tf.keras.layers.LSTMCell(rnn_size,
                                               kernel_initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.keras.layers.Dropout(cell_fw, rate=1 - keep_prob)

            cell_bw = tf.keras.layers.LSTMCell(rnn_size,
                                               kernel_initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.keras.layers.Dropout(cell_bw, rate=1 - keep_prob)
            
            encoder_cells.append(cell_fw)
            encoder_cells.append(cell_bw)

    encoder = tf.keras.layers.Bidirectional(tf.keras.layers.RNN(encoder_cells, return_sequences=True),
                                            merge_mode='concat')(rnn_inputs)
    
    return encoder


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout

def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    """Create the encoding layer"""

    encoder_cells = []
    for layer in range(num_layers):
        with tf.name_scope(f'encoder_{layer}'):
            cell_fw = tf.keras.layers.LSTMCell(rnn_size,
                                               kernel_initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = Dropout(rate=1 - keep_prob)(cell_fw)

            cell_bw = tf.keras.layers.LSTMCell(rnn_size,
                                               kernel_initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = Dropout(rate=1 - keep_prob)(cell_bw)

            encoder_cells.append(cell_fw)
            encoder_cells.append(cell_bw)

    encoder = tf.keras.layers.Bidirectional(tf.keras.layers.RNN(encoder_cells, return_sequences=True),
                                            merge_mode='concat')(rnn_inputs)

    return encoder




In [ ]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer,
                            vocab_size, max_summary_length):
    """Create the training logits"""
    
    training_helper = tf.keras.layers.RepeatVector(max_summary_length)(dec_embed_input)
    training_decoder = tf.keras.layers.RNN(dec_cell, return_sequences=True)(training_helper, initial_state=initial_state)
    training_logits = output_layer(training_decoder)
    
    return training_logits

In [ ]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    """Create the inference logits"""
    
    start_tokens = tf.tile([start_token], [batch_size])
    
    inference_helper = tf.keras.layers.GreedyEmbeddingHelper(embeddings,
                                                             start_tokens,
                                                             end_token)
    
    inference_decoder = tf.keras.layers.RNN(dec_cell, return_sequences=True)(inference_helper, initial_state=initial_state)
    inference_logits = output_layer(inference_decoder)
    
    return inference_logits


In [ ]:


def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length,
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    """Create the decoding cell and attention for the training and inference decoding layers"""

    # Create the stacked LSTM cells with dropout
    decoder_cells = []
    for layer in range(num_layers):
        lstm = tf.keras.layers.LSTMCell(rnn_size,
                                        kernel_initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        dec_cell = tf.keras.layers.Dropout(lstm, rate=1 - keep_prob)
        decoder_cells.append(dec_cell)

    # Create the output layer
    output_layer = tf.keras.layers.Dense(vocab_size,
                                         kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))

    # Create BahdanauAttention
    attn_mech = tf.keras.layers.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  use_scale=False)

    # Wrap the LSTM cell with Attention
    dec_cell = tf.keras.layers.AttentionWrapper(decoder_cells,
                                                attn_mech,
                                                rnn_size)

    # Create initial state for the decoder
    initial_state = dec_cell.get_initial_state(batch_size=batch_size, dtype=tf.float32).clone(cell_state=enc_state[0])

    # Create the training decoder
    training_decoder = tf.keras.layers.RNN(dec_cell, return_sequences=True)(dec_embed_input, initial_state=initial_state)
    training_logits = output_layer(training_decoder)

    # Create the inference decoder
    start_token = vocab_to_int['']
    end_token = vocab_to_int['']
    inference_helper = tf.keras.layers.GreedyEmbeddingHelper(embeddings, [start_token] * batch_size, end_token)
    inference_decoder = tf.keras.layers.RNN(dec_cell, return_sequences=True)(inference_helper, initial_state=initial_state)
    inference_logits = output_layer(inference_decoder)

    return training_logits, inference_logits


In [ ]:


def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length,
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    """Use the previous functions to create the training and inference logits"""

    embeddings = word_embedding_matrix

    # Convert the input_data tensor to the correct dtype (if needed)
    input_data = tf.cast(input_data, dtype=tf.int32)

    # Convert text_length tensor to correct dtype (if needed)
    text_length = tf.cast(text_length, dtype=tf.int32)

    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)

    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)

    training_logits, inference_logits = decoding_layer(dec_embed_input,
                                                       embeddings,
                                                       enc_output,
                                                       enc_state,
                                                       vocab_size,
                                                       text_length,
                                                       summary_length,
                                                       max_summary_length,
                                                       rnn_size,
                                                       vocab_to_int,
                                                       keep_prob,
                                                       batch_size,
                                                       num_layers)

    return training_logits, inference_logits



In [ ]:


def pad_sentence_batch(sentence_batch):
    """Pad sentences with  so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    padded_batch = tf.keras.preprocessing.sequence.pad_sequences(sentence_batch, 
                                                                padding='post', 
                                                                value=vocab_to_int[''],
                                                                maxlen=max_sentence)
    return padded_batch


In [ ]:


def get_batches(summaries, texts, batch_size, vocab_to_int):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        
        pad_summaries_batch = tf.keras.preprocessing.sequence.pad_sequences(summaries_batch, 
                                                                           padding='post', 
                                                                           value=vocab_to_int[''])
        pad_texts_batch = tf.keras.preprocessing.sequence.pad_sequences(texts_batch, 
                                                                        padding='post', 
                                                                        value=vocab_to_int[''])
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = [len(summary) for summary in pad_summaries_batch]
        pad_texts_lengths = [len(text) for text in pad_texts_batch]
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths


In [ ]:


# Set the Hyperparameters
epochs = 5
batch_size = 4
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75


In [ ]:

# Build the graph
train_graph = tf.Graph()

# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    # Load the model inputs
    input_data = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input')
    targets = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='targets')
    lr = tf.keras.layers.Input(shape=(), dtype=tf.float32, name='learning_rate')
    keep_prob = tf.keras.layers.Input(shape=(), dtype=tf.float32, name='keep_prob')
    summary_length = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='text_length')

    # Reverse the input data
    reversed_input_data = tf.keras.layers.Lambda(lambda x: tf.reverse(x, [-1]))(input_data)

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(reversed_input_data,
                                                      targets,
                                                      keep_prob,
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int) + 1,
                                                      rnn_size,
                                                      num_layers,
                                                      vocab_to_int,
                                                      batch_size)

    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits, name='logits')
    inference_logits = tf.identity(inference_logits, name='predictions')

    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    # Loss function
    cost = tf.keras.losses.sparse_categorical_crossentropy(targets, training_logits, from_logits=True)
    cost = tf.reduce_mean(cost * masks)

    # Optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate)

    # Apply gradients with gradient clipping
    gradients = optimizer.get_gradients(cost, trainables)
    clipped_gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
    train_op = optimizer.apply_gradients(zip(clipped_gradients, trainables))

print("Graph is built.")


In [ ]:
# Subset the data for training
start = 2000
end = start + 3000
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))
     

In [ ]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 6 #3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

  
tf.reset_default_graph()
checkpoint = "best_model.ckpt"  #300k sentence
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    # loader = tf.train.import_meta_graph(checkpoint + '.meta')
    # loader.restore(sess, checkpoint)
    #sess.run(tf.local_variables_initializer())
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0
                
                #saver = tf.train.Saver() 
                #saver.save(sess, checkpoint)
                
            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    # saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break